# Imports

In [1]:
!pip install casadi

     ---------------------------------------- 0.0/50.9 MB ? eta -:--:--
     --------------------------------------- 0.0/50.9 MB 640.0 kB/s eta 0:01:20
     --------------------------------------- 0.0/50.9 MB 660.6 kB/s eta 0:01:18
     --------------------------------------- 0.0/50.9 MB 279.3 kB/s eta 0:03:03
     --------------------------------------- 0.1/50.9 MB 476.3 kB/s eta 0:01:47
     --------------------------------------- 0.1/50.9 MB 476.3 kB/s eta 0:01:47
     --------------------------------------- 0.1/50.9 MB 476.3 kB/s eta 0:01:47
     --------------------------------------- 0.2/50.9 MB 620.6 kB/s eta 0:01:22
     --------------------------------------- 0.2/50.9 MB 655.6 kB/s eta 0:01:18
     --------------------------------------- 0.2/50.9 MB 580.1 kB/s eta 0:01:28
     --------------------------------------- 0.3/50.9 MB 655.5 kB/s eta 0:01:18
     --------------------------------------- 0.5/50.9 MB 901.1 kB/s eta 0:00:57
     --------------------------------------- 0.5


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import casadi as ca
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load Dataset and Basic Exploration
Load dataset and extract key parameters for MPC such as control limits


In [56]:
# Load dataset (Assuming CSV or already loaded DataFrame `df`)
df = pd.read_csv("C:/Users/98939/Downloads/MASTER_DATASET.csv")

# Quick look at data shape and columns
print(f"Dataset shape: {df.shape}")
# print(f"Columns: {df.columns.tolist()}")

Dataset shape: (100000, 43)


# Version 1

### Physics-Based Model (Simple Analytical Approach)

This version implements a basic physics-based model of the compressor using standard thermodynamic equations.
It is suitable for developing interpretable and analytical control strategies such as classical MPC.
The model assumes ideal conditions and provides a simplified yet explainable baseline for system behavior.


## Select states and outputs for MPC
Identify system states (e.g. Efficiency, Power_Consumption) and control input (Flow_Rate_Filtered)

In [59]:
# outputs: 'Efficiency' و 'Power_Consumption' (متغیرهایی که می‌خواهیم کنترل کنیم یا پایش کنیم)
# input: 'Flow_Rate_Filtered' (متغیر کنترل)
states = df[["Efficiency", "Power_Consumption"]].values
control_input = df["Flow_Rate_Filtered"].values

print("Sample states (Efficiency, Power Consumption):")
print(states[:5])

print("Sample control input (Flow Rate Filtered):")
print(control_input[:5])

Sample states (Efficiency, Power Consumption):
[[8.13000892e-01 5.80334527e+03]
 [8.23139087e-01 5.55471307e+03]
 [8.20195975e-01 5.65767808e+03]
 [8.19782961e-01 5.64495932e+03]
 [8.25436140e-01 5.63017483e+03]]
Sample control input (Flow Rate Filtered):
[12.15618407 12.06881673 11.96923953 11.93943977 12.00491712]


## Identify linear system matrices A and B from data
Using simple linear regression to estimate system dynamics matrices A and B

In [8]:
# Prepare data for system identification
# x_k = states[:-1]
# x_k+1 = states[1:]
# u_k = control_input[:-1]

# x_k+1 = A*x_k + B*u_k

X = np.hstack([states[:-1], control_input[:-1].reshape(-1, 1)])  # [x_k, u_k]
Y = states[1:]  # x_k+1

# remove NaN from dataset
data = np.hstack([X, Y])
data_df = pd.DataFrame(data)
data_clean = data_df.dropna()

# redefine X and Y
X_clean = data_clean.iloc[:, : X.shape[1]].values
Y_clean = data_clean.iloc[:, X.shape[1] :].values

# Fit linear regression for each state dimension separately
model = LinearRegression()
model.fit(X_clean, Y_clean)

# Extract matrices
AB = model.coef_  # shape (2, 3) because 2 outputs, 3 inputs (2 states + 1 input)
A = AB[:, : states.shape[1]]
B = AB[:, states.shape[1] :]

print("Estimated system matrix A:")
print(A)
print("\nEstimated input matrix B:")
print(B)

Estimated system matrix A:
[[ 8.87532218e-01  3.76094100e-08]
 [-8.69584738e+01  8.80242844e-01]]

Estimated input matrix B:
[[-5.70231058e-06]
 [ 4.37244845e+01]]


## Define system dimensions

In [20]:
nx = A.shape[1]  # Number of states
nu = B.shape[1]  # Number of inputs

nx, nu

(2, 1)

## Define MPC optimization problem

In [24]:
# MPC horizon
N = 10

# Define variables: states X and controls U
X = ca.MX.sym("X", nx, N + 1)  # states over horizon
U = ca.MX.sym("U", nu, N)  # inputs over horizon

X0 = ca.MX.sym("X0", nx)  # initial state (parameter)

cost = 0
g = []

Q = np.eye(nx) * 1.0
R = np.eye(nu) * 0.01

for k in range(N):
    xk = X[:, k]
    uk = U[:, k]
    xk_next = X[:, k + 1]

    # Cost function
    cost += ca.mtimes([xk.T, Q, xk]) + ca.mtimes([uk.T, R, uk])

    # System dynamics constraints
    g.append(xk_next - (ca.mtimes(A, xk) + ca.mtimes(B, uk)))

# Initial state constraint
g.append(X[:, 0] - X0)

# Flatten constraints
g = ca.vertcat(*g)

# Decision variables vector (flatten states and controls)
vars = ca.vertcat(ca.reshape(X, -1, 1), ca.reshape(U, -1, 1))

## Set up the NLP problem

In [26]:
# Define NLP structure
nlp = {
    "x": vars,  # Decision variables (all control inputs)
    "f": cost,  # Objective function (minimize cost)
    "p": X0,  # Parameters (initial state)
    "g": g,  # Constraints (dynamics)
}

## Create the IPOPT solver

In [28]:
# Define solver
solver = ca.nlpsol("solver", "ipopt", nlp)

## Solve the optimization problem

In [29]:
# Initial guess
x0 = np.zeros((nx * (N + 1) + nu * N, 1))

# Bounds on g (equality constraints)
lbg = np.zeros(g.shape)
ubg = np.zeros(g.shape)

# Parameter value (initial state)
p = states[-1]

# Solve
sol = solver(x0=x0, p=p, lbg=lbg, ubg=ubg)

# Extract solution
opt_vars = sol["x"].full().flatten()
X_opt = opt_vars[: nx * (N + 1)].reshape((nx, N + 1))
U_opt = opt_vars[nx * (N + 1) :].reshape((nu, N))

print("First optimal control input:", U_opt[:, 0])

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       82
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       40

Total number of variables............................:       32
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       22
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 4.87e+03 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

# Version 2
### Extended or Data-Driven Model (Realistic and Flexible)

This version incorporates more variables and is designed to integrate real-world sensor data for training and control.
It allows for a more realistic representation of compressor dynamics and is more suitable for machine learning-based MPC or reinforcement learning approaches.
This model offers flexibility at the cost of interpretability, making it ideal for advanced optimization tasks.


## Select Inputs and Outputs

In [50]:
# Define inputs and output
input_features = ["Pressure_In", "Temperature_In", "Flow_Rate", "Humidity"]
output_feature = "Efficiency"

# Extract inputs and output from the dataset
X = df[input_features]
y = df[output_feature]

## Normalize Data

In [51]:
# Normalize inputs and output for better performance
from sklearn.preprocessing import MinMaxScaler

input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

X_scaled = input_scaler.fit_transform(X)
y_scaled = output_scaler.fit_transform(y.values.reshape(-1, 1))

## Define Prediction Horizon (Windowing)

In [52]:
# Create input/output sequences for prediction
window_size = 10


def create_sequences(X, y, window):
    Xs, ys = [], []
    for i in range(len(X) - window):
        Xs.append(X[i : i + window])
        ys.append(y[i + window])
    return np.array(Xs), np.array(ys)


X_seq, y_seq = create_sequences(X_scaled, y_scaled, window_size)

## Train/Test Split

In [53]:
# plit data into train and test sets
split_ratio = 0.8
split = int(split_ratio * len(X_seq))

X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

## Define MPC Model with CasADi

In [54]:
# Define and solve MPC with CasADi
n_inputs = X_train.shape[2]
n_outputs = y_train.shape[1]
horizon = 10

# CasADi symbolic variables
u = ca.MX.sym("u", n_inputs)  # Inputs
y = ca.MX.sym("y", n_outputs)  # Output

# Simple linear dynamics for now (replace later with your model)
A = ca.MX.eye(n_outputs)
B = ca.MX(np.random.randn(n_outputs, n_inputs) * 0.01)

y_next = ca.mtimes(A, y) + ca.mtimes(B, u)

# Define the cost function
cost_fn = ca.Function("cost", [u, y], [ca.sumsqr(y_next - 0.5)])

## MPC Optimization Step

In [55]:
# Solve a simple MPC optimization problem for one timestep

opt_vars = ca.MX.sym("opt_vars", n_inputs)
cost = cost_fn(opt_vars, y_train[0])
nlp = {"x": opt_vars, "f": cost}
solver = ca.nlpsol("solver", "ipopt", nlp)

sol = solver(x0=np.zeros(n_inputs))
u_opt = sol["x"].full().flatten()

print("Optimal inputs:", u_opt)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0311550e-03 0.00e+00 4.93e-04  -1.0 0.00e+00    -  0.00e+00 0.00e+00 